#Tensorflow codelab MNIST

In [6]:
#parameters

BATCH_SIZE = 128
EPOCHS = 10

training_images_file = 'gs://mnist-public/train-images-idx3-ubyte'
training_labels_file = 'gs://mnist-public/train-labels-idx1-ubyte'
validation_images_file = 'gs://mnist-public/t10k-images-idx3-ubyte'
validation_labels_files = 'gs://mnist-public/t10k-labels-idx1-ubyte'

In [7]:
#imports

import os, re, math, json, shutil, pprint
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import IPython.display as display
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
print("TensorFlow version " + tf.__version__)

TensorFlow version 2.4.1


##Preparing datasets

In [18]:
#parse files, prepare training and validation datasets

AUTO = tf.data.experimental.AUTOTUNE

def read_label(tf_bytestring):
  label = tf.io.decode_raw(tf_bytestring, tf.uint8)
  label = tf.reshape(label, [])
  label = tf.one_hot(label, 10)
  return label

def read_image(tf_bytestring):
  image = tf.io.decode_raw(tf_bytestring, tf.uint8)
  image = tf.cast(image, tf.float32)/256.0
  image = tf.reshape(image, [28*28])
  return image

def load_dataset(image_file, label_file):
  imagedataset = tf.data.FixedLengthRecordDataset(image_file, 28*28, header_bytes = 16)
  imagedataset = imagedataset.map(read_image, num_parallel_calls=16)
  labelsdataset = tf.data.FixedLengthRecordDataset(label_file, 1, header_bytes=8)
  labelsdataset = labelsdataset.map(read_label, num_parallel_calls=16)
  dataset = tf.data.Dataset.zip((imagedataset, labelsdataset))
  return dataset

def get_training_dataset(image_file, label_file, batch_size):
  dataset = load_dataset(image_file, label_file)
  dataset = dataset.cache()
  dataset = dataset.shuffle(5000, reshuffle_each_iteration=True)
  dataset = dataset.repeat()
  dataset = dataset.batch(batch_size, drop_remainder=True)
  dataset = dataset.prefetch(AUTO)
  return dataset

def get_validation_dataset(image_file, label_file):
  dataset = load_dataset(image_file, label_file)
  dataset = dataset.cache()
  dataset = dataset.batch(10000, drop_remainder=True)
  dataset = dataset.repeat()
  return dataset

In [21]:
training_dataset = get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_dataset = get_validation_dataset(validation_images_file, validation_labels_files)

training_input_fn = lambda: get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_input_fn = lambda: get_validation_dataset(validation_images_file, validation_labels_files, BATCH_SIZE)

##Basic Keras Model 

In [22]:
#Keras Model

model = tf.keras.Sequential(
    [
     tf.keras.layers.Input(shape=(28*28)),
     tf.keras.layers.Dense(10, activation='softmax')
    ]
)

model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#print layers
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
          validation_data=validation_dataset, validation_steps=1)

Epoch 1/10
468/468 [==============================] - 2s 4ms/step - loss: 0.3978 - accuracy: 0.8927 - val_loss: 0.3728 - val_accuracy: 0.8995
Epoch 2/10
468/468 [==============================] - 2s 3ms/step - loss: 0.3903 - accuracy: 0.8942 - val_loss: 0.3665 - val_accuracy: 0.9012
Epoch 3/10
468/468 [==============================] - 2s 4ms/step - loss: 0.3836 - accuracy: 0.8960 - val_loss: 0.3609 - val_accuracy: 0.9025
Epoch 4/10
468/468 [==============================] - 2s 4ms/step - loss: 0.3782 - accuracy: 0.8971 - val_loss: 0.3560 - val_accuracy: 0.9037
Epoch 5/10
468/468 [==============================] - 2s 4ms/step - loss: 0.3733 - accuracy: 0.8986 - val_loss: 0.3517 - val_accuracy: 0.9049
Epoch 6/10
468/468 [==============================] - 2s 3ms/step - loss: 0.3690 - accuracy: 0.8993 - val_loss: 0.3476 - val_accuracy: 0.9062
Epoch 7/10
468/468 [==============================] - 2s 3ms/step - loss: 0.3642 - accuracy: 0.9001 - val_loss: 0.3443 - val_accuracy: 0.9071
Epoch 

In [23]:
#training and validate the model

steps_per_epoch = 60000//BATCH_SIZE
print("Steps per epoch: ", steps_per_epoch)

history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=validation_dataset, validation_steps=1)

Steps per epoch:  468
Epoch 1/10
468/468 [==============================] - 6s 9ms/step - loss: 1.6988 - accuracy: 0.5288 - val_loss: 0.8141 - val_accuracy: 0.8278
Epoch 2/10
468/468 [==============================] - 2s 4ms/step - loss: 0.7720 - accuracy: 0.8293 - val_loss: 0.6086 - val_accuracy: 0.8587
Epoch 3/10
468/468 [==============================] - 2s 4ms/step - loss: 0.6104 - accuracy: 0.8539 - val_loss: 0.5265 - val_accuracy: 0.8723
Epoch 4/10
468/468 [==============================] - 2s 3ms/step - loss: 0.5370 - accuracy: 0.8666 - val_loss: 0.4802 - val_accuracy: 0.8776
Epoch 5/10
468/468 [==============================] - 2s 4ms/step - loss: 0.4964 - accuracy: 0.8731 - val_loss: 0.4502 - val_accuracy: 0.8840
Epoch 6/10
468/468 [==============================] - 1s 3ms/step - loss: 0.4690 - accuracy: 0.8790 - val_loss: 0.4287 - val_accuracy: 0.8884
Epoch 7/10
468/468 [==============================] - 2s 3ms/step - loss: 0.4408 - accuracy: 0.8851 - val_loss: 0.4123 - val_a

In [26]:
probabilities = model.predict(validation_dataset, steps=1)
predicted_labels = np.argmax(probabilities, axis=1)

##Adding Layers to the model

In [30]:
#Adding Layers

model = tf.keras.Sequential(
    [
     tf.keras.layers.Input(shape=(28*28)),
     tf.keras.layers.Dense(200, activation='sigmoid'),
     tf.keras.layers.Dense(60, activation='sigmoid'),
     tf.keras.layers.Dense(10, activation='softmax')
    ]
)

In [32]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_9 (Dense)              (None, 60)                12060     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                610       
Total params: 169,670
Trainable params: 169,670
Non-trainable params: 0
_________________________________________________________________


In [33]:
#training and validate the model

steps_per_epoch = 60000//BATCH_SIZE
print("Steps per epoch: ", steps_per_epoch)

history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=validation_dataset, validation_steps=1)

Steps per epoch:  468
Epoch 1/10
468/468 [==============================] - 2s 4ms/step - loss: 2.3127 - accuracy: 0.1609 - val_loss: 2.2317 - val_accuracy: 0.3968
Epoch 2/10
468/468 [==============================] - 2s 3ms/step - loss: 2.2162 - accuracy: 0.3833 - val_loss: 2.1556 - val_accuracy: 0.5188
Epoch 3/10
468/468 [==============================] - 2s 4ms/step - loss: 2.1360 - accuracy: 0.5118 - val_loss: 2.0492 - val_accuracy: 0.5985
Epoch 4/10
468/468 [==============================] - 2s 3ms/step - loss: 2.0211 - accuracy: 0.5884 - val_loss: 1.8999 - val_accuracy: 0.6196
Epoch 5/10
468/468 [==============================] - 2s 3ms/step - loss: 1.8646 - accuracy: 0.6240 - val_loss: 1.7114 - val_accuracy: 0.6708
Epoch 6/10
468/468 [==============================] - 2s 3ms/step - loss: 1.6733 - accuracy: 0.6634 - val_loss: 1.5090 - val_accuracy: 0.6978
Epoch 7/10
468/468 [==============================] - 2s 3ms/step - loss: 1.4731 - accuracy: 0.6908 - val_loss: 1.3218 - val_a

##Special care for vanishing gradient problem by adding RELU

In [35]:
model = tf.keras.Sequential([
                             tf.keras.layers.Input(shape=(28*28, )),
                             tf.keras.layers.Dense(200, activation='relu'),
                             tf.keras.layers.Dense(60, activation='relu'),
                             tf.keras.layers.Dense(10, activation='softmax')
])

In [36]:
#using adam optimizer instead of sgd

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 200)               157000    
_________________________________________________________________
dense_14 (Dense)             (None, 60)                12060     
_________________________________________________________________
dense_15 (Dense)             (None, 10)                610       
Total params: 169,670
Trainable params: 169,670
Non-trainable params: 0
_________________________________________________________________


In [38]:
steps_per_epoch = 6000 // BATCH_SIZE
print("Steps per epoch: ", steps_per_epoch)

history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, 
                    validation_data = validation_dataset, validation_steps=1)

Steps per epoch:  46
Epoch 1/10
46/46 [==============================] - 1s 26ms/step - loss: 1.4807 - accuracy: 0.5597 - val_loss: 0.4110 - val_accuracy: 0.8784
Epoch 2/10
46/46 [==============================] - 1s 16ms/step - loss: 0.3924 - accuracy: 0.8816 - val_loss: 0.3124 - val_accuracy: 0.9111
Epoch 3/10
46/46 [==============================] - 1s 15ms/step - loss: 0.3070 - accuracy: 0.9119 - val_loss: 0.2594 - val_accuracy: 0.9260
Epoch 4/10
46/46 [==============================] - 1s 16ms/step - loss: 0.2549 - accuracy: 0.9292 - val_loss: 0.2214 - val_accuracy: 0.9365
Epoch 5/10
46/46 [==============================] - 1s 17ms/step - loss: 0.2295 - accuracy: 0.9312 - val_loss: 0.2011 - val_accuracy: 0.9397
Epoch 6/10
46/46 [==============================] - 1s 15ms/step - loss: 0.2184 - accuracy: 0.9369 - val_loss: 0.1802 - val_accuracy: 0.9462
Epoch 7/10
46/46 [==============================] - 1s 15ms/step - loss: 0.1972 - accuracy: 0.9419 - val_loss: 0.1796 - val_accuracy:

##Learning rate decay by tweaking optimizer params

In [40]:
model = tf.keras.Sequential([
                             tf.keras.layers.Input(shape=(28*28, )),
                             tf.keras.layers.Dense(200, activation='relu'),
                             tf.keras.layers.Dense(60, activation='relu'),
                             tf.keras.layers.Dense(10, activation='softmax')
])

#increasing the learning rate from 0.001 to 0.01
optimizer = tf.keras.optimizers.Adam(lr=0.01)

In [41]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 200)               157000    
_________________________________________________________________
dense_17 (Dense)             (None, 60)                12060     
_________________________________________________________________
dense_18 (Dense)             (None, 10)                610       
Total params: 169,670
Trainable params: 169,670
Non-trainable params: 0
_________________________________________________________________


In [43]:
steps_per_epoch = 60000 // BATCH_SIZE
print("steps per epoch: ", steps_per_epoch)

def lr_decay(epoch):
  return 0.01 * math.pow(0.6, epoch)

lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=True)

history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, 
                    epochs=EPOCHS, validation_data=validation_dataset, validation_steps=1)

steps per epoch:  468
Epoch 1/10
468/468 [==============================] - 2s 4ms/step - loss: 0.3775 - accuracy: 0.8799 - val_loss: 0.1322 - val_accuracy: 0.9587
Epoch 2/10
468/468 [==============================] - 2s 4ms/step - loss: 0.1237 - accuracy: 0.9629 - val_loss: 0.1221 - val_accuracy: 0.9653
Epoch 3/10
468/468 [==============================] - 2s 4ms/step - loss: 0.0960 - accuracy: 0.9710 - val_loss: 0.1082 - val_accuracy: 0.9694
Epoch 4/10
468/468 [==============================] - 2s 4ms/step - loss: 0.0913 - accuracy: 0.9718 - val_loss: 0.1166 - val_accuracy: 0.9696
Epoch 5/10
468/468 [==============================] - 2s 4ms/step - loss: 0.0777 - accuracy: 0.9768 - val_loss: 0.1515 - val_accuracy: 0.9650
Epoch 6/10
468/468 [==============================] - 2s 4ms/step - loss: 0.0678 - accuracy: 0.9799 - val_loss: 0.1353 - val_accuracy: 0.9683
Epoch 7/10
468/468 [==============================] - 2s 4ms/step - loss: 0.0624 - accuracy: 0.9813 - val_loss: 0.1450 - val_a

##Dropout

In [46]:
#adding dropout of 25%

model = tf.keras.Sequential([
                             tf.keras.Input(shape=(28*28, )),
                             tf.keras.layers.Dense(200, activation='relu'),
                             tf.keras.layers.Dropout(0.25),
                             tf.keras.layers.Dense(100, activation='relu'),
                             tf.keras.layers.Dropout(0.25),
                             tf.keras.layers.Dense(60, activation='relu'),
                             tf.keras.layers.Dropout(0.25),
                             tf.keras.layers.Dense(10, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 200)               157000    
_________________________________________________________________
dropout_6 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 60)                6060      
_________________________________________________________________
dropout_8 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 10)               

In [47]:
steps_per_epoch=60000 // BATCH_SIZE
print("Steps per epoch: ", steps_per_epoch)

def lr_decay(epoch):
  return 0.01 * math.pow(0.6, epoch)

lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=True)

history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, 
                    validation_data=validation_dataset, validation_steps=1, callbacks=[lr_decay_callback])

Steps per epoch:  468
Epoch 1/10
468/468 [==============================] - 3s 5ms/step - loss: 0.5968 - accuracy: 0.8187 - val_loss: 0.1745 - val_accuracy: 0.9522
Epoch 2/10
468/468 [==============================] - 2s 4ms/step - loss: 0.2722 - accuracy: 0.9287 - val_loss: 0.1713 - val_accuracy: 0.9549
Epoch 3/10
468/468 [==============================] - 2s 4ms/step - loss: 0.2500 - accuracy: 0.9350 - val_loss: 0.1415 - val_accuracy: 0.9597
Epoch 4/10
468/468 [==============================] - 2s 4ms/step - loss: 0.2378 - accuracy: 0.9398 - val_loss: 0.1459 - val_accuracy: 0.9627
Epoch 5/10
468/468 [==============================] - 2s 4ms/step - loss: 0.2219 - accuracy: 0.9438 - val_loss: 0.1520 - val_accuracy: 0.9638
Epoch 6/10
468/468 [==============================] - 2s 4ms/step - loss: 0.2152 - accuracy: 0.9473 - val_loss: 0.1666 - val_accuracy: 0.9648
Epoch 7/10
468/468 [==============================] - 2s 4ms/step - loss: 0.1955 - accuracy: 0.9499 - val_loss: 0.1583 - val_a

##Convolutional Networks

In [52]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Reshape(input_shape=(28*28, ), target_shape=(28, 28, 1)),
     tf.keras.layers.Conv2D(kernel_size=3, filters=12, activation='relu', padding='same'),
     tf.keras.layers.Conv2D(kernel_size=6, filters=24, activation='relu', padding='same', strides=2),
     tf.keras.layers.Conv2D(kernel_size=6, filters=32, activation='relu', padding='same', strides=2),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(200, activation='relu'),
     tf.keras.layers.Dense(10, activation='softmax')

    ]
)

optimizer=tf.keras.optimizers.Adam(lr=0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 12)        120       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 24)        10392     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          27680     
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 200)               313800    
_________________________________________________________________
dense_32 (Dense)             (None, 10)              

In [53]:
def lr_decay(epoch):
  return 0.01 * math.pow(0.6, epoch)

  lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=True)

  plot_learning_rate(lr_decay, EPOCHS)

In [55]:
steps_per_epoch = 60000//BATCH_SIZE
print("steps per epoch: ", steps_per_epoch)

history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, 
                    validation_data=validation_dataset, validation_steps=1, 
                    callbacks=[lr_decay_callback])

steps per epoch:  468
Epoch 1/10

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
468/468 [==============================] - 9s 8ms/step - loss: 0.3676 - accuracy: 0.8830 - val_loss: 0.0728 - val_accuracy: 0.9752
Epoch 2/10

Epoch 00002: LearningRateScheduler reducing learning rate to 0.006.
468/468 [==============================] - 3s 6ms/step - loss: 0.0494 - accuracy: 0.9848 - val_loss: 0.0514 - val_accuracy: 0.9853
Epoch 3/10

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0036.
468/468 [==============================] - 3s 6ms/step - loss: 0.0248 - accuracy: 0.9922 - val_loss: 0.0346 - val_accuracy: 0.9898
Epoch 4/10

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0021599999999999996.
468/468 [==============================] - 3s 6ms/step - loss: 0.0112 - accuracy: 0.9966 - val_loss: 0.0373 - val_accuracy: 0.9902
Epoch 5/10

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001296.
468/468 [==========================

##Batch Normalization



In [58]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1)),
     tf.keras.layers.Conv2D(kernel_size=3, filters=12, use_bias=False, padding='same'),
     tf.keras.layers.BatchNormalization(center=True, scale=False),
     tf.keras.layers.Activation('relu'),
     tf.keras.layers.Conv2D(kernel_size=6, filters=24, use_bias=False, padding='same', strides=2),
     tf.keras.layers.BatchNormalization(center=True, scale=False),
     tf.keras.layers.Activation('relu'),
     tf.keras.layers.Conv2D(kernel_size=6, filters=32, use_bias=False, padding='same', strides=2),
     tf.keras.layers.BatchNormalization(center=True, scale=False),
     tf.keras.layers.Activation('relu'),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(200, use_bias=False),
     tf.keras.layers.BatchNormalization(center=True, scale=False),
     tf.keras.layers.Activation('relu'),
     tf.keras.layers.Dropout(0.3),
     tf.keras.layers.Dense(10, activation='softmax')
    ]
)

optimizer=tf.keras.optimizers.Adam(lr=0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 12)        108       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 12)        36        
_________________________________________________________________
activation (Activation)      (None, 28, 28, 12)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 24)        10368     
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 24)        72        
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 24)      

In [59]:
steps_per_epoch = 60000//BATCH_SIZE
print("steps per epoch: ", steps_per_epoch)

history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, 
                    validation_data=validation_dataset, validation_steps=1, 
                    callbacks=[lr_decay_callback])

steps per epoch:  468
Epoch 1/10

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
468/468 [==============================] - 4s 8ms/step - loss: 0.2327 - accuracy: 0.9299 - val_loss: 0.0636 - val_accuracy: 0.9798
Epoch 2/10

Epoch 00002: LearningRateScheduler reducing learning rate to 0.006.
468/468 [==============================] - 3s 6ms/step - loss: 0.0439 - accuracy: 0.9864 - val_loss: 0.0570 - val_accuracy: 0.9805
Epoch 3/10

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0036.
468/468 [==============================] - 3s 7ms/step - loss: 0.0259 - accuracy: 0.9922 - val_loss: 0.0239 - val_accuracy: 0.9912
Epoch 4/10

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0021599999999999996.
468/468 [==============================] - 3s 6ms/step - loss: 0.0161 - accuracy: 0.9947 - val_loss: 0.0235 - val_accuracy: 0.9919
Epoch 5/10

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001296.
468/468 [==========================